In [4]:
# Import Modules
from tensorflow.keras.layers import *
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import string
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
from numpy import array, argmax
import os
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWiMePfwFnvq_P_f53TMQndtqywWczMhXx33ZbTL8Wi51zDIZrZTbY8


In [3]:
cur_dir = os.path.abspath(os.path.dirname('__file__'))
cur_dir = os.path.join(cur_dir, 'drive')
cur_dir = os.path.join(cur_dir, 'MyDrive')
cur_dir = os.path.join(cur_dir, '2021SMP')
cur_dir = os.path.join(cur_dir, 'SMP 21RV03')
cur_dir = os.path.join(cur_dir, 'FL_Server_Client')
model_dir = os.path.join(cur_dir,  'models')
# model_dir

In [ ]:
def get_data(filename, number):
    # JP: number=732190
    # ZJ: number=643821
    # RS: number=670068
    
    file = open(filename, "r", encoding="utf-8")
    length = 63500
    counter = 0
    idx = 0
    texts = []
    new_texts = [] # lines of text data
    for row in file:
        texts.append(row)
        idx += 1
        if idx == length:
            break
    for text in texts:
        text = text.replace("\n", "").replace("\r", "").replace('\ufeff', '')
        punctuation_to_space = str.maketrans(string.punctuation, " "*len(string.punctuation))
        new_text = text.translate(punctuation_to_space)
        new_texts.append(new_text)
        counter += 1
        print("{}%".format(counter/length * 100), end="\r")
    data = ""
    counter = 0
    for new in new_texts:
        data = " ".join(new_texts)
        counter += 1
        print("{}%".format(counter/length * 100), end="\r")

    data = ' '.join(data.split(' ')[:-4])

    
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([data[:number]])

    # convert to tf-idf vector form
    pickle.dump(tokenizer, open('client_tokenizer.pkl', 'wb'))

    sequence_data = tokenizer.texts_to_sequences([data[:number]])[0]
    vocab_size = len(tokenizer.word_index)+1
    sequences = []
    for i in range(1, len(sequence_data)):
        words = sequence_data[i-1:i+1]
        sequences.append(words)

    sequences = np.array(sequences)

    x = []
    y = []
    for i in sequences:
        x.append(i[0])
        y.append(i[1])
    x = np.array(x)
    y = np.array(y)
    y = to_categorical(y, num_classes=vocab_size)

    print(x.shape, y.shape)
    return x, y
    # 10692 is the vocab size

In [ ]:
def train(model, x, y, direc):
    checkpoint = ModelCheckpoint(direc, monitor="loss", verbose=1,\
                            save_best_only=True, mode="auto")
    reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=1, min_lr=0.0001, verbose = 1)
    logdir='logsnextword'
    tensorboard_Visualization = TensorBoard(log_dir=logdir)
    model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.1), metrics=["accuracy"])
    history = model.fit(x, y, epochs=20, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

    print('-'*200)
    print(direc, ':')
    print('loss: ', history.history['loss'])
    print('accuracy: ', history.history['accuracy'])
    with open(os.path.join(cur_dir, 'record.txt'), 'a') as f:
      f.write(direc+'\n')
      f.write('loss: {}(min)\n'.format(min(history.history['loss'])))
      f.write('acc: {}(max)\n\n'.format(min(history.history['accuracy'])))
    
    model.save(direc)
    return None

In [ ]:
import math
def get_weight(cycle_num):
  weight = -1/2 * (math.e ** (-cycle_num)) + 1
  return weight
get_weight(99)

1.0

In [ ]:
for count in range(36, 41):
    ser_model = load_model(os.path.join(model_dir, ('nextword_server_'+str(count-1)+'.h5')))
    jp_model = load_model(os.path.join(model_dir, ('nextword_jp_'+str(count-1)+'.h5')))
    zj_model = load_model(os.path.join(model_dir, ('nextword_zj_'+str(count-1)+'.h5')))
    rs_model = load_model(os.path.join(model_dir, ('nextword_rs_'+str(count-1)+'.h5')))
    
    weights = [ser_model.get_weights(), jp_model.get_weights(), zj_model.get_weights(), rs_model.get_weights()]
        
    avg_weight = []
    for i in range(len(weights[0])):
        for j in range(len(weights)):
            if j == 0:
                array = weights[j][i]
            else:
                array = array + weights[j][i]
        array = array / len(weights)
        avg_weight.append(array)
        
    ser_model.set_weights(avg_weight)
    ser_model.save(os.path.join(model_dir, ('nextword_server_'+str(count)+'.h5')))
    
    print('Averaged Server Model {} saved'.format(count))
    
    for i in range(1, len(weights)):
        weight = [avg_weight, weights[i]]
        avg_weight = []
        
        for j in range(len(weight[0])):
            w = get_weight(count)
            array = weight[0] * (1-w) + weight[1] * (w)
            array /= 2
            avg_weight.append(array)
        
        if i == 1:
            jp_model.set_weights(avg_weight)
            x, y = get_data(os.path.join(cur_dir, 'client_data_jp.txt'), 732190)
            # x, y = get_data(jp_data, 732190)
            train(jp_model, x, y, os.path.join(model_dir, ('nextword_jp_'+str(count)+'.h5')))
            x, y = 0, 0
        elif i == 2:
            zj_model.set_weights(avg_weight)
            x, y = get_data(os.path.join(cur_dir, 'client_data_zj.txt'), 643821)
            # x, y = get_data(zj_data, 643821)
            train(zj_model, x, y, os.path.join(model_dir, ('nextword_zj_'+str(count)+'.h5')))
            x, y = 0, 0
        elif i == 3:
            rs_model.set_weights(avg_weight)
            x, y = get_data(os.path.join(cur_dir, 'client_data_rs.txt'), 670068)
            # x, y = get_data(rs_data, 670068)
            train(rs_model, x, y, os.path.join(model_dir, ('nextword_rs_'+str(count)+'.h5')))
            x, y = 0, 0

Averaged Server Model 36 saved
(150179,) (150179, 10692)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
2347/2347 [==============================] - 48s 17ms/step - loss: 7.2499 - accuracy: 0.0276

Epoch 00001: loss improved from inf to 7.24988, saving model to /content/drive/MyDrive/2021SMP/SMP 21RV03/FL_Server_Client/models/nextword_jp_36.h5
Epoch 2/20
2347/2347 [==============================] - 39s 17ms/step - loss: 7.0264 - accuracy: 0.0273

Epoch 00002: loss improved from 7.24988 to 7.02635, saving model to /content/drive/MyDrive/2021SMP/SMP 21RV03/FL_Server_Client/models/nextword_jp_36.h5
Epoch 3/20
2347/2347 [==============================] - 39s 17ms/step - loss: 7.0087 - accuracy: 0.0273

Epoch 00003: loss improved from 7.02635 to 7.00874, saving model to /content/drive/MyDrive/2021SMP/SMP 21RV03/FL_Server_Client/models/nextword_jp_36.h5
Epoch 4/20
2347/2347 [==============================] - 39s 17ms/step - loss: 7.0083 - accuracy: 0.0276

Epoch 00004: loss improved from 7.00874 to 7.00826, saving model to /content/drive/MyDrive/2021SMP/SMP 21RV03/FL_Server_Client/m